# Test Hydra Config

In [9]:
# Load Hydra config in notebooks
# https://github.com/facebookresearch/hydra/blob/main/examples/jupyter_notebooks/compose_configs_in_notebook.ipynb
import os
from hydra import initialize_config_dir, compose
import hydra
from omegaconf import OmegaConf

abs_config_dir = os.path.abspath("config/")

with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
    config = compose(config_name="main.yaml", overrides=[])
    # print(OmegaConf.to_yaml(env_config))
    print(config)


{'device': 'cpu', 'float_precision': 32, 'budget': 5, 'n_samples': 3, 'seed': 31415, 'maximize': False, 'dataset': {'grid_size': 10, 'normalize_scores': True, 'train_fraction': 1.0, 'batch_size': 16, 'shuffle': True, 'train_path': '${user.data.root}/branin/data_10_train.csv', 'test_path': None, '_target_': 'activelearning.dataset.grid.BraninDatasetHandler'}, 'oracle': {'_target_': 'activelearning.oracle.oracle.BraninOracle', 'fidelity': 1, 'do_domain_map': True}, 'selector': {'_target_': 'activelearning.selector.selector.Selector'}, 'sampler': {'_target_': 'activelearning.sampler.sampler.GreedySampler'}, 'surrogate': {'_target_': 'activelearning.surrogate.gp_surrogate.SingleTaskGPRegressor'}, 'acquisition': {'_target_': 'activelearning.acquisition.acquisition.BOTorchMaxValueEntropyAcquisition', 'acq_fn_class': {'_target_': 'botorch.acquisition.max_value_entropy_search.qLowerBoundMaxValueEntropy', '_partial_': True}}, 'user': {'logdir': {'root': './logs'}, 'data': {'root': './data'}}, '

In [ ]:

dataset = hydra.utils.instantiate(
    config.dataset,
    float_precision=config.float_precision,
)

oracle = hydra.utils.instantiate(
    config.oracle,
    device=config.device,
    float_precision=config.float_precision,
)

surrogate = hydra.utils.instantiate(
    config.surrogate,
    device=config.device,
    float_precision=config.float_precision,
)

acquisition = hydra.utils.instantiate(
    config.acquisition,
    surrogate_model=surrogate,
    dataset_handler=dataset,
    device=config.device,
    float_precision=config.float_precision,
)

sampler = hydra.utils.instantiate(
    config.sampler,
    acquisition=acquisition,
    device=config.device,
    float_precision=config.float_precision,
)

selector = hydra.utils.instantiate(
    config.selector,
    score_fn=acquisition.get_acquisition_values,
    device=config.device,
    float_precision=config.float_precision,
)

{'device': 'cpu', 'float_precision': 32, 'budget': 5, 'n_samples': 3, 'seed': 31415, 'maximize': False, 'dataset': {'grid_size': 10, 'normalize_scores': True, 'train_fraction': 1.0, 'batch_size': 16, 'shuffle': True, 'train_path': '${user.data.root}/branin/data_10_train.csv', 'test_path': None, '_target_': 'activelearning.dataset.grid.BraninDatasetHandler'}, 'oracle': {'_target_': 'activelearning.oracle.oracle.Branin', 'fidelity': 1, 'do_domain_map': True}, 'selector': {'_target_': 'activelearning.selector.selector.Selector'}, 'sampler': {'_target_': 'activelearning.sampler.sampler.GreedySampler'}, 'surrogate': {'_target_': 'activelearning.surrogate.gp_surrogate.SingleTaskGPRegressor'}, 'acquisition': {'_target_': 'activelearning.acquisition.acquisition.BOTorchMaxValueEntropyAcquisition', 'acq_fn_class': {'_target_': 'botorch.acquisition.max_value_entropy_search.qLowerBoundMaxValueEntropy', '_partial_': True}}, 'user': {'logdir': {'root': './logs'}, 'data': {'root': './data'}}, 'logger

### Create Larger Branin Set

In [1]:
from activelearning.dataset.grid import BraninDatasetHandler
from activelearning.oracle.oracle import BraninOracle


# Dataset
dataset_handler = BraninDatasetHandler(
    grid_size=100,
    train_path="./data/branin/data_%i_train.csv" % 100,
    train_fraction=1.0,
    float_precision=32,
)

candidate_set, xi, yi = dataset_handler.get_candidate_set()

# Oracle
oracle = BraninOracle(
    fidelity=1, do_domain_map=True, device="cpu", float_precision=32
)


In [2]:
# uniform sampling
import numpy as np
subset_idcs = np.random.uniform(0,1,size=(len(candidate_set)))>0.97
subset = candidate_set[subset_idcs]
from activelearning.oracle.oracle import BraninOracle
oracle = BraninOracle(fidelity=1, device="cpu", float_precision=32)

dataset_handler.update_dataset(subset, oracle(subset.clone()), "data/branin/large_branin_100.csv")

(<torch.utils.data.dataloader.DataLoader at 0x7f710c53d630>, None)

In [21]:
# quadrant sampling
import numpy as np
candidate_q1 = candidate_set[(candidate_set[:,0]<0.5) * (candidate_set[:,1]<0.5)]
candidate_q2 = candidate_set[(candidate_set[:,0]<0.5) * (candidate_set[:,1]>=0.5)]
candidate_q3 = candidate_set[(candidate_set[:,0]>=0.5) * (candidate_set[:,1]<0.5)]
candidate_q4 = candidate_set[(candidate_set[:,0]>=0.5) * (candidate_set[:,1]>=0.5)]

subset_idcs_1 = np.random.uniform(0,1,size=(len(candidate_q1)))>0.99
subset_idcs_2 = np.random.uniform(0,1,size=(len(candidate_q1)))>0.99
subset_idcs_3 = np.random.uniform(0,1,size=(len(candidate_q1)))>0.99
subset_idcs_4 = np.random.uniform(0,1,size=(len(candidate_q1)))>1.0

subset_1 = candidate_q1[subset_idcs_1]
subset_2 = candidate_q2[subset_idcs_2]
subset_3 = candidate_q3[subset_idcs_3]
subset_4 = candidate_q4[subset_idcs_4]

import torch
subset = torch.concat([subset_1, subset_2, subset_3, subset_4])
subset.shape

torch.Size([74, 2])

In [22]:
from activelearning.oracle.oracle import BraninOracle
oracle = BraninOracle(fidelity=1, device="cpu", float_precision=32)

dataset_handler.update_dataset(subset, oracle(subset.clone()), "data/branin/large_branin_quadrant_100.csv")

(<torch.utils.data.dataloader.DataLoader at 0x7f8dd0bdd0f0>, None)

# Active Learning Loop

In [1]:
# Load Hydra config in notebooks
# https://github.com/facebookresearch/hydra/blob/main/examples/jupyter_notebooks/compose_configs_in_notebook.ipynb
import os
from hydra import initialize_config_dir, compose
import hydra
from omegaconf import OmegaConf
abs_config_dir = os.path.abspath("config/")

with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
    config = compose(config_name="test_branin.yaml", overrides=[])
    print(OmegaConf.to_yaml(config))
    print(config)

config.sampler.conf.logger.do.online = False

/home/mila/c/christina.humer/.conda/envs/al_new/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'test_branin.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


dataset:
  train_fraction: 1.0
  batch_size: 16
  shuffle: true
  train_path: ~/activelearning/data/branin/data_100_train.csv
  test_path: null
  _target_: activelearning.dataset.grid.BraninDatasetHandler
env:
  _target_: gflownet.envs.grid.Grid
  env_id: env
  fixed_distr_params: null
  random_distr_params: null
  skip_mask_check: false
  conditional: false
  continuous: false
  id: grid
  func: corners
  n_dim: 2
  length: 100
  max_increment: 1
  max_dim_per_action: 1
  cell_min: -1
  cell_max: 1
  buffer:
    train: null
    test: null
oracle:
  _target_: activelearning.oracle.oracle.BraninOracle
  fidelity: 1
  do_domain_map: true
  negate: false
sampler:
  conf:
    policy:
      forward:
        _target_: gflownet.policy.base.Policy
        config:
          type: mlp
          n_hid: 2048
          n_layers: 2
          checkpoint: null
          reload_ckpt: false
          is_model: false
      backward:
        _target_: gflownet.policy.base.Policy
        config: null
     

In [2]:
import torch
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = config.device
n_iterations = config.budget  # TODO: replace with budget
grid_size = config.env.length
n_samples = config.n_samples

from gflownet.utils.common import set_float_precision
float_prec = set_float_precision(config.float_precision)
# float_prec = set_float_precision(32)

import matplotlib.colors as cm
import matplotlib.pyplot as plt
# colors = ["red", "blue", "green", "orange", "brown", "pink"]
colors = plt.get_cmap("Reds")

In [3]:
from activelearning.dataset.grid import BraninDatasetHandler
from activelearning.surrogate.gp_surrogate import (
    SingleTaskGPRegressor,
    DeepKernelSVGPSurrogate,
)
from activelearning.acquisition.acquisition import BOTorchMaxValueEntropyAcquisition
from activelearning.sampler.sampler import GreedySampler, RandomSampler
from activelearning.selector.selector import Selector, ScoreSelector
from activelearning.oracle.oracle import BraninOracle
from activelearning.surrogate.feature_extractor.mlp import MLP, Identity
from gflownet.envs.grid import Grid as GridEnv
from functools import partial

# Environment
env_maker = partial(GridEnv, n_dim=2, length=grid_size)

# Dataset
dataset_handler = BraninDatasetHandler(
    env=env_maker(),
    train_path="./data/branin/data_%i_train.csv" % grid_size,
    train_fraction=1.0,
    float_precision=float_prec,
)

candidate_set, xi, yi = dataset_handler.get_candidate_set()

from activelearning.utils.logger import WandBLogger, ConsoleLogger

# logger = WandBLogger(project_name="test_branin", run_name="100_greedy_gp_mve_fix-selector") # "100_continuous-gflownet_gp_mve" # "100_greedy_dkl-30epochs_mve"
logger = ConsoleLogger(project_name="test_branin", run_name="Greedy 10x10")
# logger = None


from activelearning.utils.plotter import PlotHelper, BraninCIME4RExportHelper

plotter = PlotHelper()
# plotter = BraninCIME4RExportHelper(dataset_handler)

# Oracle
oracle = BraninOracle(
    fidelity=1, do_domain_map=True, device=device, float_precision=float_prec
)

if plotter is not None:
    fig_oracle, ax_oracle = plotter.plot_function(
        oracle, candidate_set[:], xi=xi, yi=yi, label="oraclefn"
    )


best_scores = []
all_scores = {}

for i in range(n_iterations):

    train_data, test_data = dataset_handler.get_dataloader()
    # print("iteration", i)
    # Surrogate (e.g., Bayesian Optimization)
    # starts with a clean slate each iteration
    surrogate = SingleTaskGPRegressor(
        device=device,
        float_precision=float_prec,
    )

    # feature_extractor = Identity(2)
    # feature_extractor = MLP(
    #     n_input=2,
    #     n_hidden=[16, 16],
    #     n_output=7,
    #     float_precision=32,
    # )
    # surrogate = DeepKernelSVGPSurrogate(
    #     feature_extractor,
    #     float_precision=float_prec,
    #     device=device,
    #     mll_args={"num_data": len(train_data.dataset)},
    #     train_epochs=30,
    #     lr=0.1,
    #     logger=logger,
    # )
    surrogate.fit(train_data)

    acq_fn = BOTorchMaxValueEntropyAcquisition(
        surrogate.model, device=device, float_precision=float_prec
    )

    # Sampler (e.g., GFlowNet, or Random Sampler)
    # also starts with a clean slate; TODO: experiment with NOT training from scratch
    # sampler = RandomSampler(acq_fn)
    # sampler = GreedySampler(
    #     acq_fn,
    #     device=device,
    #     float_precision=float_prec,
    # )
    sampler = hydra.utils.instantiate(
        config.sampler,
        env_maker=env_maker,
        acquisition=acq_fn,
        device=device,
        float_precision=float_prec,
        _recursive_=False,
    )

    sampler.fit()  # only necessary for samplers that train a model

    samples, sample_idcs = sampler.get_samples(
        n_samples * 5, candidate_set=candidate_set
    )

    if plotter is not None and hasattr(sampler, "sampler"):

        def reward_fn(samples):
            return sampler.sampler.proxy.proxy2reward(sampler.sampler.proxy(samples))

        fig_reward, ax_reward = plotter.plot_function(
            reward_fn,
            candidate_set[:],
            xi=xi,
            yi=yi,
            label="rewardfn",
            iteration=i,
        )
        fig_reward, ax_reward = plotter.plot_samples(torch.Tensor(samples), ax_reward, fig_reward)
        ax_reward.set_title("reward fn + proposed samples of iteration %i" % i)
        logger.log_figure(fig_reward, "reward")

    # Selector
    # selector = Selector(
    #     device=device,
    #     float_precision=float_prec,
    # )
    selector = ScoreSelector(
        acq_fn,
        device=device,
        float_precision=float_prec,
        maximize=True,
    )
    filtered_samples, selected_idcs = selector(n_samples=n_samples, candidate_set=dataset_handler.get_custom_dataset(torch.Tensor(samples)), index_set=sample_idcs)
    filtered_samples = torch.Tensor(filtered_samples)

    if plotter is not None:
        x, y = train_data.dataset.get_raw_item()
        fig_surrogate, ax_surrogate = plt.subplots(ncols=2, figsize=(15, 7))
        plotter.plot_function(
            surrogate.get_predictions,
            candidate_set[:],
            xi=xi,
            yi=yi,
            output_index=0,
            fig=fig_surrogate,
            ax=ax_surrogate[0],
            label="pred_target_mean",
            iteration=i,
        )
        plotter.plot_samples(x, ax=ax_surrogate[0], fig=fig_surrogate, targets=y)
        ax_surrogate[0].set_title("predicted mean + train data of iteration %i" % i)
        plotter.plot_function(
            surrogate.get_predictions,
            candidate_set[:],
            xi=xi,
            yi=yi,
            output_index=1,
            fig=fig_surrogate,
            ax=ax_surrogate[1],
            label="pred_target_var",
            iteration=i,
        )
        plotter.plot_samples(x, ax=ax_surrogate[1], fig=fig_surrogate, targets=y)
        ax_surrogate[1].set_title("predicted var + train data of iteration %i" % i)
        logger.log_figure(fig_surrogate, "surrogate")

    if plotter is not None:
        fig_acq, ax_acq = plotter.plot_function(
            acq_fn,
            candidate_set[:],
            xi=xi,
            yi=yi,
            label="acq",
            iteration=i,
        )
        fig_acq, ax_acq = plotter.plot_samples(filtered_samples, ax_acq, fig_acq)
        ax_acq.set_title("acquisition fn + selected samples of iteration %i" % i)
        logger.log_figure(fig_acq, "acq")

    if plotter is not None:
        ax_oracle.scatter(
            x=filtered_samples[:, 1].cpu(),
            y=filtered_samples[:, 0].cpu(),
            c=cm.to_hex(colors(i / n_iterations)),
            marker="x",
            label="it %i" % i,
        )
    
    scores = oracle(dataset_handler.get_custom_dataset(filtered_samples)[:]).cpu()
    scores = dataset_handler.update_dataset(filtered_samples, scores)
    best_scores.append(scores.max())
    all_scores[i] = scores
    if logger is not None:
        scores_flat = torch.stack(list(all_scores.values())).flatten()
        logger.log_metric(scores_flat.max(), "top_score")
        mean_top_k = (scores_flat
            .topk(n_samples, largest=True)
            .values.mean()
        )
        logger.log_metric(mean_top_k, "mean_topk_score")
        logger.log_metric(scores.max(), "best_score")
        logger.log_step(i)

    if plotter is not None:
        plotter.plot_scores(selected_idcs=selected_idcs, scores=scores, i=i+1)

if ax_oracle is not None:
    fig_oracle.legend()
    ax_oracle.set_title("oracle fn + samples")
    logger.log_figure(fig_oracle, key="oracle")

if plotter is not None:
    plotter.end(filename="branin.csv")

fig = plt.figure()
plt.plot(best_scores)
plt.xlabel("iterations")
plt.ylabel("scores")
plt.title("Best Score in each iteration")
if logger is not None:
    logger.log_figure(fig, key="best_scores")
    logger.end()

/home/mila/c/christina.humer/.conda/envs/al_new/lib/python3.10/site-packages/botorch/models/gp_regression.py:161: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  self._validate_tensor_args(X=transformed_X, Y=train_Y, Yvar=train_Yvar)
/home/mila/c/christina.humer/.conda/envs/al_new/lib/python3.10/site-packages/botorch/models/utils/assorted.py:174: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)



            Important: offline trajectories will NOT be sampled. In order to sample
            offline trajectories, the train configuration of the buffer should be
            complete and feasible. It should at least specify env.buffer.train.type.
            

            Important: test metrics will NOT be computed. In order to compute
            test metrics the test configuration of the buffer should be complete and
            feasible. It should at least specify env.buffer.test.type.
            


  0%|          | 0/500 [00:00<?, ?it/s]/home/mila/c/christina.humer/gflownet-dev/gflownet/utils/buffer.py:141: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.main = pd.concat(
Loss: 1.6406 | Mean rewards: 0.12 | JSD: -1.0000:  85%|████████▌ | 425/500 [03:38<00:56,  1.33it/s]  

In [4]:
train_data.dataset[:]

(tensor([[-1.0000, -0.9798],
         [-0.9798, -0.1717],
         [-0.1313, -1.0000],
         [ 0.6768,  0.6970]]),
 tensor([1.0000, 0.3512, 0.0000, 0.4139]))

In [5]:
next(iter(train_data))

[tensor([[[-0.1313, -1.0000]],
 
         [[ 0.6768,  0.6970]],
 
         [[-0.9798, -0.1717]],
 
         [[-1.0000, -0.9798]]]),
 tensor([0.0000, 0.4139, 0.3512, 1.0000])]

In [10]:
len(train_data.dataset[0][0][0].shape)

1

# Test individual components

In [14]:
# Load Hydra config in notebooks
# https://github.com/facebookresearch/hydra/blob/main/examples/jupyter_notebooks/compose_configs_in_notebook.ipynb
import os
from hydra import initialize_config_dir, compose
import hydra
from omegaconf import OmegaConf
abs_config_dir = os.path.abspath("config/")

with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
    config = compose(config_name="test_branin.yaml", overrides=[])
    print(OmegaConf.to_yaml(config))
    print(config)

config.sampler.conf.logger.do.online = False

/home/mila/c/christina.humer/.conda/envs/al_new/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'test_branin.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


dataset:
  grid_size: 100
  normalize_scores: true
  train_fraction: 1.0
  batch_size: 16
  shuffle: true
  train_path: ~/activelearning/data/branin/data_100_train.csv
  test_path: null
  _target_: activelearning.dataset.grid.BraninDatasetHandler
oracle:
  _target_: activelearning.oracle.oracle.BraninOracle
  fidelity: 1
  do_domain_map: true
  negate: false
sampler:
  conf:
    env:
      _target_: gflownet.envs.grid.Grid
      env_id: env
      fixed_distr_params: null
      random_distr_params: null
      skip_mask_check: false
      conditional: false
      continuous: false
      buffer:
        replay_capacity: 0
        train: null
        test: null
      id: grid
      func: corners
      n_dim: 2
      length: 100
      max_increment: 1
      max_dim_per_action: 1
      cell_min: -1
      cell_max: 1
    policy:
      forward:
        _target_: gflownet.policy.base.Policy
        config:
          type: mlp
          n_hid: 2048
          n_layers: 2
          checkpoint: nul

In [15]:
import torch
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = config.device
n_iterations = config.budget  # TODO: replace with budget
grid_size = config.env.length
n_samples = config.n_samples

from gflownet.utils.common import set_float_precision
float_prec = set_float_precision(config.float_precision)
# float_prec = set_float_precision(32)


In [16]:
from activelearning.dataset.grid import BraninDatasetHandler
from activelearning.surrogate.gp_surrogate import SingleTaskGPRegressor
from activelearning.acquisition.acquisition import BOTorchMaxValueEntropyAcquisition
from activelearning.oracle.oracle import BraninOracle

# Dataset
dataset_handler = BraninDatasetHandler(
    grid_size=100,
    train_path="./data/branin/data_%i_train.csv" % 100,
    train_fraction=1.0,
    float_precision=float_prec,
)

train_data, test_data = dataset_handler.get_dataloader()
# Surrogate (e.g., Bayesian Optimization)
# starts with a clean slate each iteration
surrogate = SingleTaskGPRegressor(
    device=device,
    float_precision=float_prec,
)
surrogate.fit(train_data)

acq_fn = BOTorchMaxValueEntropyAcquisition(
    surrogate.model, device=device, float_precision=float_prec
)

# Oracle
oracle = BraninOracle(
    fidelity=1, do_domain_map=True, device=device, float_precision=float_prec
)

/home/mila/c/christina.humer/.conda/envs/al_new/lib/python3.10/site-packages/botorch/models/gp_regression.py:161: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  self._validate_tensor_args(X=transformed_X, Y=train_Y, Yvar=train_Yvar)
/home/mila/c/christina.humer/.conda/envs/al_new/lib/python3.10/site-packages/botorch/models/utils/assorted.py:174: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)


In [17]:
config.sampler.conf

{'env': {'_target_': 'gflownet.envs.grid.Grid', 'env_id': 'env', 'fixed_distr_params': None, 'random_distr_params': None, 'skip_mask_check': False, 'conditional': False, 'continuous': False, 'buffer': {'replay_capacity': 0, 'train': None, 'test': None}, 'id': 'grid', 'func': 'corners', 'n_dim': 2, 'length': 100, 'max_increment': 1, 'max_dim_per_action': 1, 'cell_min': -1, 'cell_max': 1}, 'policy': {'forward': {'_target_': 'gflownet.policy.base.Policy', 'config': {'type': 'mlp', 'n_hid': 2048, 'n_layers': 2, 'checkpoint': None, 'reload_ckpt': False, 'is_model': False}}, 'backward': {'_target_': 'gflownet.policy.base.Policy', 'config': None}, 'shared': None}, 'agent': {'_target_': 'gflownet.gflownet.GFlowNetAgent', 'seed': 0, 'optimizer': {'z_dim': 16, 'loss': 'trajectorybalance', 'lr': 0.0005, 'lr_decay_period': 1000000, 'lr_decay_gamma': 0.5, 'lr_z_mult': 20, 'method': 'adam', 'early_stopping': 0.0, 'ema_alpha': 0.5, 'adam_beta1': 0.9, 'adam_beta2': 0.999, 'sgd_momentum': 0.9, 'batch_s

In [18]:
from gflownet.envs.grid import Grid

In [19]:
config.sampler.conf.env

{'_target_': 'gflownet.envs.grid.Grid', 'env_id': 'env', 'fixed_distr_params': None, 'random_distr_params': None, 'skip_mask_check': False, 'conditional': False, 'continuous': False, 'buffer': {'replay_capacity': 0, 'train': None, 'test': None}, 'id': 'grid', 'func': 'corners', 'n_dim': 2, 'length': 100, 'max_increment': 1, 'max_dim_per_action': 1, 'cell_min': -1, 'cell_max': 1}

In [21]:

# Sampler (e.g., GFlowNet, or Random Sampler)
sampler = hydra.utils.instantiate(
    config.sampler,
    acquisition=acq_fn,
    device=device,
    float_precision=float_prec,
    _recursive_=False,
)

In [5]:
sampler.fit()  # only necessary for samplers that train a model

  0%|          | 0/100 [00:00<?, ?it/s]/home/mila/c/christina.humer/.conda/envs/al_new/lib/python3.10/site-packages/gflownet/utils/buffer.py:140: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.main = pd.concat(
Loss: 2095.3161 | Mean rewards: -0.08 | JSD: -1.0000: 100%|██████████| 100/100 [00:09<00:00, 10.41it/s]


In [8]:
samples, sample_idcs = sampler.get_samples(
    n_samples * 5, candidate_set=None
)
samples

tensor([[0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0100, 0.0000],
        [0.0000, 0.0100],
        [0.0100, 0.0000],
        [0.0100, 0.0000],
        [0.0100, 0.0000],
        [0.0200, 0.0000],
        [0.0000, 0.0200],
        [0.0100, 0.0100],
        [0.0000, 0.0200],
        [0.0000, 0.0200],
        [0.0100, 0.0200],
        [0.0100, 0.0200]])

### Grid Environment

In [1]:
from gflownet.envs.grid import Grid

In [2]:
grid_env = Grid(2, 10, 1, 1, -1, 1)
grid_env.action_space

[(0, 1), (1, 0), (0, 0)]

In [4]:
grid_env.states2proxy([[0, 0], [0, 4], [4, 0], [8, 8]])

[[0, 0], [0, 4], [4, 0], [8, 8]]
tensor([[0., 0.],
        [0., 4.],
        [4., 0.],
        [8., 8.]])


tensor([[-1.0000, -1.0000],
        [-1.0000, -0.1111],
        [-0.1111, -1.0000],
        [ 0.7778,  0.7778]])

In [6]:
grid_env.cells

array([-1.        , -0.77777778, -0.55555556, -0.33333333, -0.11111111,
        0.11111111,  0.33333333,  0.55555556,  0.77777778,  1.        ])

### deprecated

In [ ]:
import os

# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"
print(device)


from gflownet.utils.common import set_float_precision
float_prec = set_float_precision(64)
# float_prec = torch.double

import numpy as np
grid_size = 10  # 100
test_states = torch.tensor([[0.5, 0.5]])
test_scores = torch.tensor([-150.0])
n_samples = 5

baseline_states = torch.tensor(
    [
        [0.0 * grid_size, 0.0 * grid_size],
        [0.0 * grid_size, 0.4 * grid_size],
        [0.4 * grid_size, 0.0 * grid_size],
        [0.8 * grid_size, 0.8 * grid_size],
    ]
)

# define grid space
xi = np.arange(0, 1, 1 / grid_size)
yi = np.arange(0, 1, 1 / grid_size)
grid = np.array(np.meshgrid(xi, yi))
grid_flat = torch.tensor(grid.T, dtype=float_prec).reshape(-1, 2).to(device)

In [ ]:
# dataset
from dataset.dataset import BraninDatasetHandler

dataset_handler = BraninDatasetHandler(
    grid_size=grid_size,
    train_path="./storage/branin/data_%i_train.csv" % grid_size,
    train_fraction=1.0,
    float_precision=float_prec,
)

# testing dataset
dataset_handler.update_dataset(test_states, test_scores)
print(dataset_handler.train_data.X_data, dataset_handler.train_data.y_data)
print(dataset_handler.train_data[:])
train_loader, test_loader = dataset_handler.get_dataloader()
for X, y in train_loader:
    print(X, y)

In [ ]:
# Oracle
from gflownet.proxy.box.branin import Branin

oracle = Branin(fidelity=1, do_domain_map=True, device=device, float_precision=float_prec)

# testing oracle
print(oracle(baseline_states.to(device).clone() / grid_size))

# plot oracle function
plot_function(oracle, grid_flat.clone(), scatter_markers=baseline_states / grid_size)
plt.title("Oracle function")
plt.show()

In [ ]:
# Surrogate (e.g., Bayesian Optimization)
from surrogate.surrogate import SingleTaskGPRegressor

surrogate = SingleTaskGPRegressor(device=device, float_precision=float_prec)
train_data, test_data = dataset_handler.get_dataloader()
surrogate.fit(train_data)

# testing surrogate
preds_mean, preds_var = surrogate.get_predictions(test_states / grid_size)
print(preds_mean, preds_var)

# plot surrogate functions
fig, axes = plt.subplots(ncols=3, figsize=(15, 4))

plot_function(
    surrogate.get_predictions,
    grid_flat.clone(),
    fig=fig,
    ax=axes[0],
    scatter_markers=baseline_states / grid_size,
    output_index=0,
)
axes[0].set_title("estimated mean")

plot_function(
    surrogate.get_predictions,
    grid_flat.clone(),
    fig=fig,
    ax=axes[1],
    scatter_markers=baseline_states / grid_size,
    output_index=1,
)
axes[1].set_title("uncertainty")

plot_function(
    surrogate.get_acquisition_values,
    grid_flat.clone(),
    fig=fig,
    ax=axes[2],
    scatter_markers=baseline_states / grid_size,
)
axes[2].set_title("acquisition function")

fig.suptitle("Surrogate Model")
plt.show()

In [ ]:
# Sampler (e.g., GFlowNet, or Random Sampler)
from sampler.sampler import GreedySampler, RandomSampler

sampler = GreedySampler(surrogate)
# sampler = RandomSampler(surrogate)
sampler.fit()
samples, _ = sampler.get_samples(n_samples * 3, grid_flat.clone()).cpu()

# plot acq function with proposed candidates
fig, ax = plot_function(
    surrogate.get_acquisition_values, grid_flat.clone(), scatter_markers=samples
)
plt.title("Samples")
plt.show()

In [ ]:
acq = surrogate.get_acquisition_values(grid_flat.clone()).detach().cpu()
plt.matshow(acq.reshape(int(len(acq) ** (1 / 2)), int(len(acq) ** (1 / 2))))
plt.scatter(samples.cpu()[:, 1] * grid_size, samples.cpu()[:, 0] * grid_size)
plt.scatter(baseline_states[:, 1].cpu(), baseline_states[:, 0].cpu())

In [ ]:
from filter.filter import Filter, ScoreFilter

filter = Filter()
# filter = OracleFilter(oracle)
filtered_samples = filter(n_samples, samples)

# plot oracle function with proposed candidates
fig, ax = plot_function(
    surrogate.get_acquisition_values,
    grid_flat.clone(),
    scatter_markers=filtered_samples.to("cpu"),
)
plt.title("Proposed Candidates")
plt.show()

# GFlowNet Sampler

In [ ]:
import torch
float_prec = torch.float32
device = "cpu"
grid_size = 10

In [ ]:
# dataset
from dataset.dataset import BraninDatasetHandler

dataset_handler = BraninDatasetHandler(
    grid_size=grid_size,
    train_path="./storage/branin/data_%i_train.csv" % grid_size,
    train_fraction=1.0,
    float_precision=float_prec,
)


In [ ]:
train_data, _ = dataset_handler.get_dataloader()
for r in train_data:
    print(r)

In [ ]:
from importlib import reload  # Python 3.4+
import surrogate.surrogate as surrogate

reload(surrogate)

from surrogate.surrogate import SingleTaskGPRegressor

surrogate = SingleTaskGPRegressor(device=device, float_precision=float_prec, maximize=False)
train_data, test_data = dataset_handler.get_dataloader()
surrogate.fit(train_data)

In [ ]:
# Load Hydra config in notebooks
# https://github.com/facebookresearch/hydra/blob/main/examples/jupyter_notebooks/compose_configs_in_notebook.ipynb
import os
from hydra import initialize_config_dir, compose
import hydra
from omegaconf import OmegaConf

abs_config_dir = os.path.abspath("config/")


In [ ]:
with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
    config_logger = compose(config_name="logger/wandb.yaml", overrides=[])
    # print(OmegaConf.to_yaml(env_config))
    print(config_logger)

logger = hydra.utils.instantiate(config_logger.logger, config_logger, _recursive_=False)

In [ ]:
with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
    config_logger = compose(config_name="logger/wandb.yaml", overrides=[])
    # print(OmegaConf.to_yaml(env_config))
    print(config_logger)

# Logger
# from utils.logger import Logger

# logger = Logger(config=OmegaConf.create(), **config_logger)
logger = hydra.utils.instantiate(config_logger.logger, config_logger, _recursive_=False)

with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
    config_sampler = compose(config_name="sampler/gflownet.yaml", overrides=[])
    # print(OmegaConf.to_yaml(config_sampler))
    print(config_sampler)

grid_env = hydra.utils.instantiate(
    config_sampler.sampler.conf.env,
    proxy=surrogate,
    device=device,
    float_precision=float_prec,
)

# The policy is used to model the probability of a forward/backward action
forward_policy = hydra.utils.instantiate(
    config_sampler.sampler.conf.policy.forward,
    env=grid_env,
    device=device,
    float_precision=float_prec,
)
backward_policy = hydra.utils.instantiate(
    config_sampler.sampler.conf.policy.backward,
    env=grid_env,
    device=device,
    float_precision=float_prec,
)

# State flow
if config_sampler.sampler.conf.state_flow is not None:
    state_flow = hydra.utils.instantiate(
        config_sampler.sampler.conf.state_flow,
        env=grid_env,
        device=device,
        float_precision=float_prec,
        base=forward_policy,
    )
else:
    state_flow = None

# GFlowNet Agent
sampler = hydra.utils.instantiate(
    config_sampler.sampler.conf.agent,
    device=device,
    float_precision=float_prec,
    env=grid_env,
    forward_policy=forward_policy,
    backward_policy=backward_policy,
    state_flow=state_flow,
    buffer=config_sampler.sampler.conf.env.buffer,
    logger=logger,
)

In [ ]:
sampler.train()

In [ ]:
batch, times = sampler.sample_batch(n_forward=10, train=False)

In [ ]:
batch.get_terminating_states(proxy=True)

In [ ]:
sampler

In [ ]:
with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
    config_sampler = compose(config_name="sampler/gflownet.yaml", overrides=[])
    # print(OmegaConf.to_yaml(config_sampler))
    print(config_sampler)
    
sampler = hydra.utils.instantiate(
    config_sampler.sampler,
    surrogate=surrogate,
    logger=logger,
    device=device,
    float_precision=float_prec,
)


In [ ]:
samples, _ = sampler.get_samples(n_samples * 3, grid_flat.clone()).cpu()

# plot acq function with proposed candidates
fig, ax = plot_function(
    surrogate.get_acquisition_values, grid_flat.clone(), scatter_markers=samples
)
plt.title("Samples")
plt.show()

In [ ]:
from sampler.sampler import GFlowNetSampler

sampler = GFlowNetSampler(surrogate, config_sampler, logger, device, float_prec)

In [ ]:

# Logger
# from utils.logger import Logger

# logger = Logger(config=OmegaConf.create(), **config_logger)

In [ ]:
from gflownet.envs.grid import Grid as GFlowNetGridEnv

grid_env = GFlowNetGridEnv(
    n_dim=2,
    length=grid_size,
    max_increment=1,
    max_dim_per_action=1,
    cell_min=0,
    cell_max=1,
    proxy=surrogate,
    reward_func="identity",
)

In [ ]:
# Load Hydra config in notebooks
# https://github.com/facebookresearch/hydra/blob/main/examples/jupyter_notebooks/compose_configs_in_notebook.ipynb
import os
from hydra import initialize_config_dir, compose
import hydra
from omegaconf import OmegaConf

abs_config_dir = os.path.abspath("config/")
with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
    config_gflownet = compose(config_name="gflownet.yaml", overrides=[])
    # print(OmegaConf.to_yaml(env_config))
    print(config_gflownet)

# GFlowNetAgent
from gflownet.policy.base import Policy
from gflownet.gflownet import GFlowNetAgent

forward_policy = Policy(
    config_gflownet.policy.forward,
    env=grid_env,
    device=device,
    float_precision=torch.double,
)

backward_policy = Policy(
    config_gflownet.policy.backward,
    env=grid_env,
    device=device,
    float_precision=torch.double,
)

sampler = GFlowNetAgent(
    grid_env,
    device=device,
    float_precision=torch.double,
    forward_policy=forward_policy,
    backward_policy=backward_policy,
    logger=logger,
    **config_gflownet
)

In [ ]:
sampler.train()

In [ ]:
from gflownet.utils.batch import Batch
batch = Batch(env=grid_env, device=device, float_type=float_prec)

In [ ]:
# Make indices of batch consecutive since they are used for indexing here
# Get necessary tensors from batch
states_policy = batch.get_states(policy=True)
states = batch.get_states(policy=False)
actions = batch.get_actions()
parents_policy = batch.get_parents(policy=True)
parents = batch.get_parents(policy=False)
traj_indices = batch.get_trajectory_indices(consecutive=True)
# Forward trajectories
masks_f = batch.get_masks_forward(of_parents=True)
policy_output_f = forward_policy(parents_policy)

In [ ]:
# # environment
# import config
# train = config.env.BufferSpec(
#     path="~/scratch/datasets/grid/corners_d2l3_r0.csv",
#     seed=167,
#     n=1000,
# )
# test = config.env.BufferSpec(
#     path="~/scratch/datasets/grid/corners_d2l3_r0.csv",
#     seed=167,
#     n=1000,
# )
# buffer = config.env.Buffer(train=train, test=test)
# config_env = config.env.Grid_Env(
#     buffer,
#     min_step_len=1,
#     max_step_len=1,
#     cell_min=-1,
#     cell_max=1
# )

# from environment.grid_env import GridEnv
# env = GridEnv(**config_env.__dict__)